In [ ]:
!pip install tensorflow matplotlib seaborn scikit-learn

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2, InceptionV3, EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import os, zipfile
print("TensorFlow version:", tf.__version__)

In [ ]:
# ================================================
# 2. Upload & Prepare Dataset
# ================================================
from google.colab import files
uploaded = files.upload()  # Upload your fish dataset zip

In [ ]:
# Unzip dataset
zip_path = '/content/Dataset.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('data')

In [ ]:
!ls data

In [ ]:
train_dir = '/content/data/images.cv_jzk6llhf18tm3k0kyttxz/data/test'
val_dir = '/content/data/images.cv_jzk6llhf18tm3k0kyttxz/data/val'
tesr_dir='/content/data/images.cv_jzk6llhf18tm3k0kyttxz/data/train'

In [ ]:
# Image size (224 for most models, 299 for InceptionV3)
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [ ]:
# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

NUM_CLASSES = train_gen.num_classes
test_gen = val_datagen.flow_from_directory(
    tesr_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)
NUM_CLASSES = test_gen.num_classes

In [ ]:
# ================================================
# 3. Helper Functions
# ================================================
def plot_history(history, title):
    plt.figure(figsize=(12,4))
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label='train acc')
    plt.plot(history.history['val_accuracy'], label='val acc')
    plt.title(f'{title} - Accuracy')
    plt.legend()
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label='train loss')
    plt.plot(history.history['val_loss'], label='val loss')
    plt.title(f'{title} - Loss')
    plt.legend()
    plt.show()
def evaluate_model(model, val_gen, model_name):
    val_gen.reset()
    preds = model.predict(val_gen, verbose=0)
    y_pred = np.argmax(preds, axis=1)
    y_true = val_gen.classes
    target_names = list(val_gen.class_indices.keys())

    print(f"=== {model_name} ===")
    print(classification_report(y_true, y_pred, target_names=target_names))

    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_names, yticklabels=target_names)
    plt.title(f'{model_name} - Confusion Matrix')
    plt.show()

    accuracy = np.mean(y_pred == y_true)
    return accuracy


In [ ]:
# ================================================
# 4. CNN from Scratch
# ================================================
def build_cnn(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, 3, activation='relu')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3, activation='relu')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inputs, outputs)
cnn_model = build_cnn((*IMG_SIZE,3), NUM_CLASSES)
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = [
    ModelCheckpoint('cnn_best.h5', monitor='val_accuracy', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3),
    EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
]

history_cnn = cnn_model.fit(train_gen, validation_data=val_gen, epochs=20, callbacks=callbacks)
plot_history(history_cnn, "CNN from Scratch")
acc_cnn = evaluate_model(cnn_model, val_gen, "CNN from Scratch")

In [ ]:
# ================================================
# 5. Transfer Learning (Fast Version - MobileNetV2)
# ================================================

from tensorflow.keras.applications import MobileNetV2

print("\n=== Training MobileNetV2 (with fine-tuning) ===")

# Make sure image size matches MobileNetV2
IMG_SIZE = (224, 224)
train_gen = train_datagen.flow_from_directory(train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')
val_gen = val_datagen.flow_from_directory(val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False)

# Load base model
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(*IMG_SIZE,3))
base_model.trainable = False  # Freeze base model initially

# Build model
inputs = layers.Input(shape=(*IMG_SIZE,3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)
model = models.Model(inputs, outputs)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = [
    ModelCheckpoint('MobileNetV2_best.h5', monitor='val_accuracy', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2),
    EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
]

# Stage 1: Train frozen base
history = model.fit(train_gen, validation_data=val_gen, epochs=3, callbacks=callbacks)
plot_history(history, "MobileNetV2 - Frozen Base")

# Stage 2: Fine-tune last few layers
base_model.trainable = True
for layer in base_model.layers[:-30]:  # Keep most layers frozen
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
history_ft = model.fit(train_gen, validation_data=val_gen, epochs=5, callbacks=callbacks)
plot_history(history_ft, "MobileNetV2 - Fine-tuned")

acc_mobilenet = evaluate_model(model, val_gen, "MobileNetV2 Fine-tuned")


In [ ]:
# ================================================
# 6. Model Comparison (Updated for Deployment)
# ================================================
from tensorflow.keras.models import load_model
cnn_model = load_model('/content/cnn_best.h5')
acc_cnn = evaluate_model(cnn_model, val_gen, "CNN from Scratch")
results = []
results.append(("CNN from Scratch", acc_cnn))
df_results = pd.DataFrame(results, columns=['Model', 'Validation Accuracy'])
df_results = df_results.sort_values(by='Validation Accuracy', ascending=False)
print(df_results)

plt.figure(figsize=(8,5))
sns.barplot(x='Validation Accuracy', y='Model', data=df_results)
plt.title('Model Accuracy Comparison')
plt.show()

# Save the best model name for reference
best_model_name = df_results.iloc[0]['Model']
print(f"Best model: {best_model_name}")

# If you saved each model as '{ModelName}_best.h5', you can find the file
best_model_path = f"{best_model_name}_best.h5"
print(f"Best model file path: {best_model_path}")


In [ ]:
# ================================================
# 7. Test on Uploaded Image (Updated)
# ================================================

# Load your best trained model directly
model_path = "cnn_best.h5"  # or your best pre-trained model file
final_model = tf.keras.models.load_model(model_path)

# Upload an image for testing
from google.colab import files
uploaded_img = files.upload()

# Get image path and preprocess
img_path = list(uploaded_img.keys())[0]
img = tf.keras.utils.load_img(img_path, target_size=IMG_SIZE)
img_array = tf.keras.utils.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predict
pred = final_model.predict(img_array)
pred_class = np.argmax(pred)

# Use the same class labels from your training generator
labels = list(train_gen.class_indices.keys())

print(f"Prediction: {labels[pred_class]} | Confidence: {np.max(pred):.2f}")
